# Agenda, day 3 — real-world data

1. Recap + Q&A
2. More on CSV
3. Reading online data
4. Sorting
5. Grouping
6. Pivot tables
7. Joining tables
8. Cleaning data

# Recap

1. Most work in Pandas is done in a data frame
    - 2D table
    - Columns -- names must be unique
    - Rows -- with an index that doesn't need to be unique
2. We can retrieve (or assign) to a Pandas data frame using `.loc` and `.iloc`
3. In particular, using `.loc` is a key part of working with Pandas
    - In the one-argument version, we just pass a *row selector*, aka `df.loc[ROW_SELECTOR]`, which can be:
        - A string (for the index of the row(s) we want)
        - A list of strings (for the indexes of the row(s) we want)
        - A boolean series (indicating which rows we want, wherever there's a `True` in there)
        - A slice, for a number of rows
    - In the two-argument version, we pass both a *row selector* and a *column selector*, separated by a comma, which looks like: `df.loc[ROW_SELECTOR, COLUMN SELECTOR]`. The row selector is the same as the one-argument version, but the column selector is also the same, and can be used to describe which columns we want. 
4. At the end of last week, we saw that we can read data from a CSV file    

# Reading from CSV files

"CSV" stands for "comma-separated values" or "character-separated values."  The idea is:

- Every row in the file represents one record
- The field separator (a comma, by default) separates fields from one another in each record.

You can imagine a CSV file looking something like this:

```
United States,English
United Kingdom,English
France,French
Germany,German
Netherlands,Dutch
```

If the above is a CSV file, then we want to read it into a data frame, such that the country will be one column and the language will be a second column.

There is a problem with what I did here: There is no header line, giving the column names! If we aren't careful, then when we read it in, the first line will be seen as the column names, not as a data row.

Let's consider this, then:

```
country,language
United States,English
United Kingdom,English
France,French
Germany,German
Netherlands,Dutch
```

If we want to read a CSV file into a data frame, we use `pd.read_csv`. This function takes a filename as an argument, and assumes a lot of defaults for *many* options that can be passed.  Here are a few of the options we might want to pass to `pd.read_csv`.

(Note that all of these are passed as keyword arguments, meaning that they are of the form `name=value` in the function call.)

1. `sep` -- pass this to indicate what character is separating the fields on each line. By default, it's `','`. I often use `'\t'` (tab) for a separator, because I find it easier to read.
2. `header` -- pass this to indicate on what line of the file the header is located. This is useful if you want to skip over several rows, until you get to the column names or the data. If there are no column names (i.e., no header), then just pass `header=None`.
3. `usecols` -- this argument takes a list of strings (or of integers, if you really want), indicating which fields should be included in the data frame. If you pass strings, then they should be column names. If you pass integers, they should be column indexes, starting with 0.
4. `dtype` -- this argument should be a dictionary of column names and types that we want to assign. This way, you can force Pandas's hands.

In [10]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

In [8]:
!ls *.csv

burrito_current.csv	   languages.csv  titanic3.csv
celebrity_deaths_2016.csv  taxi.csv


In [9]:
df = pd.read_csv('taxi.csv')

In [3]:
!head taxi.csv

In [4]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2015-06-02 11:19:29,2015-06-02 11:47:52,1,1.63,-73.954430,40.764141,1,N,-73.974754,40.754093,2,17.0,0.0,0.5,0.00,0.0,0.3,17.80
1,2,2015-06-02 11:19:30,2015-06-02 11:27:56,1,0.46,-73.971443,40.758942,1,N,-73.978539,40.761909,1,6.5,0.0,0.5,1.00,0.0,0.3,8.30
2,2,2015-06-02 11:19:31,2015-06-02 11:30:30,1,0.87,-73.978111,40.738434,1,N,-73.990273,40.745438,1,8.0,0.0,0.5,2.20,0.0,0.3,11.00
3,2,2015-06-02 11:19:31,2015-06-02 11:39:02,1,2.13,-73.945892,40.773529,1,N,-73.971527,40.760330,1,13.5,0.0,0.5,2.86,0.0,0.3,17.16
4,1,2015-06-02 11:19:32,2015-06-02 11:32:49,1,1.40,-73.979088,40.776772,1,N,-73.982162,40.758999,2,9.5,0.0,0.5,0.00,0.0,0.3,10.30


In [5]:
df = pd.read_csv('taxi.csv', usecols=['trip_distance', 'total_amount', 'passenger_count'])

In [6]:
df.head()

,passenger_count,trip_distance,total_amount
0,1,1.63,17.80
1,1,0.46,8.30
2,1,0.87,11.00
3,1,2.13,17.16
4,1,1.40,10.30


In [7]:
df.dtypes  # what dtype is each column?

passenger_count      int64
trip_distance      float64
total_amount       float64
dtype: object

In [11]:
df = pd.read_csv('taxi.csv', 
                 usecols=['trip_distance', 'total_amount', 'passenger_count'],
                dtype={'trip_distance':np.float64,
                      'total_amount':np.float64,
                      'passenger_count':np.int8})

In [12]:
df.dtypes

passenger_count       int8
trip_distance      float64
total_amount       float64
dtype: object

In [13]:
help(pd.read_csv)

Help on function read_csv in module pandas.io.parsers.readers:

read_csv(filepath_or_buffer: 'FilePath | ReadCsvBuffer[bytes] | ReadCsvBuffer[str]', *, sep: 'str | None | lib.NoDefault' = <no_default>, delimiter: 'str | None | lib.NoDefault' = None, header: "int | Sequence[int] | None | Literal['infer']" = 'infer', names: 'Sequence[Hashable] | None | lib.NoDefault' = <no_default>, index_col: 'IndexLabel | Literal[False] | None' = None, usecols=None, dtype: 'DtypeArg | None' = None, engine: 'CSVEngine | None' = None, converters=None, true_values=None, false_values=None, skipinitialspace: 'bool' = False, skiprows=None, skipfooter: 'int' = 0, nrows: 'int | None' = None, na_values=None, keep_default_na: 'bool' = True, na_filter: 'bool' = True, verbose: 'bool' = False, skip_blank_lines: 'bool' = True, parse_dates: 'bool | Sequence[Hashable] | None' = None, infer_datetime_format: 'bool | lib.NoDefault' = <no_default>, keep_date_col: 'bool' = False, date_parser=<no_default>, date_format: 'str

# Exercise: Short and long taxi rides

1. Load `taxi.csv` into a data frame. You want the following columns: `passenger_count`, `trip_distance`, `total_amount`, `tip_amount`.
2. What percentage of taxi riders never tip? (That is, zero tip?)
3. What percentage, on average, do they tip? (That is: All tips / all total amounts)

In [14]:
df = pd.read_csv('taxi.csv',
                usecols=['passenger_count', 'trip_distance', 'total_amount', 'tip_amount'])
df.head()

,passenger_count,trip_distance,tip_amount,total_amount
0,1,1.63,0.00,17.80
1,1,0.46,1.00,8.30
2,1,0.87,2.20,11.00
3,1,2.13,2.86,17.16
4,1,1.40,0.00,10.30


In [16]:
# what percentage of taxi riders never tip? tip is 0

# the value_counts method tallies up all of the values in a series, and tells us how many times each appears
(df['tip_amount'] == 0).value_counts()

tip_amount
False    5729
True     4270
Name: count, dtype: int64

In [18]:
# what percentage for each? 
# doesn't this strike you as a bit ... long?
(df['tip_amount'] == 0).value_counts() / df['tip_amount'].count()

tip_amount
False    0.572957
True     0.427043
Name: count, dtype: float64

In [19]:
# let's just get the percentages!
(df['tip_amount'] == 0).value_counts(normalize=True)

tip_amount
False    0.572957
True     0.427043
Name: proportion, dtype: float64

In [20]:
# what percentage do people tip, on average?

df['tip_amount'].sum() / df['total_amount'].sum()

0.1035785033739647

In [29]:
# want to know how many rows are in a data frame? Use len() on the index!
len(df.loc[df['tip_amount'] == 0].index)

4270

# Finding files

When I load a CSV file (or any other file) into Pandas, I can give it a few types of filenames. (I'm going to use Unix paths here, which use `/`. If you're on Windows, then just use `\\` instead, or a raw string with a single `\`.)

- If a filename contains no `/` characters, it's assumed to be in the current directory.
- If it contains a `/`, but doesn't start with one, then it's assumed to be in a subdirectory. For example, `abc/def.csv` is the `def.csv` file in the `abc` subdirectory.
- If it *starts* with a `/`, then it's an absolute path on your filesystem.

In this case, I just wrote `taxi.csv`, because the file is in the same directory as where I'm running Jupyter.

# What about files that are online?

We just saw that -- we can download `taxi.csv` onto our filesystem, and then load it with `pd.read_csv`.

What if we could shorten that?

You can pass a URL to read_csv, and it takes all of the other options that we've discussed.

In [31]:
pd.read_csv('https://raw.githubusercontent.com/reuven/oreilly-2023-05May-data/main/taxi.csv')

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2015-06-02 11:19:29,2015-06-02 11:47:52,1,1.63,-73.954430,40.764141,1,N,-73.974754,40.754093,2,17.0,0.0,0.5,0.00,0.0,0.3,17.80
1,2,2015-06-02 11:19:30,2015-06-02 11:27:56,1,0.46,-73.971443,40.758942,1,N,-73.978539,40.761909,1,6.5,0.0,0.5,1.00,0.0,0.3,8.30
2,2,2015-06-02 11:19:31,2015-06-02 11:30:30,1,0.87,-73.978111,40.738434,1,N,-73.990273,40.745438,1,8.0,0.0,0.5,2.20,0.0,0.3,11.00
3,2,2015-06-02 11:19:31,2015-06-02 11:39:02,1,2.13,-73.945892,40.773529,1,N,-73.971527,40.760330,1,13.5,0.0,0.5,2.86,0.0,0.3,17.16
4,1,2015-06-02 11:19:32,2015-06-02 11:32:49,1,1.40,-73.979088,40.776772,1,N,-73.982162,40.758999,2,9.5,0.0,0.5,0.00,0.0,0.3,10.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,1,2015-06-01 00:12:59,2015-06-01 00:24:18,1,2.70,-73.947792,40.814972,1,N,-73.973358,40.783638,2,11.0,0.5,0.5,0.00,0.0,0.3,12.30
9995,1,2015-06-01 00:12:59,2015-06-01 00:28:16,1,4.50,-74.004066,40.747818,1,N,-73.953758,40.779285,1,16.0,0.5,0.5,3.00,0.0,0.3,20.30
9996,2,2015-06-01 00:13:00,2015-06-01 00:37:25,1,5.59,-73.994377,40.766102,1,N,-73.903206,40.750546,2,21.0,0.5,0.5,0.00,0.0,0.3,22.30
9997,2,2015-06-01 00:13:02,2015-06-01 00:19:10,6,1.54,-73.978302,40.748531,1,N,-73.989166,40.762852,2,6.5,0.5,0.5,0.00,0.0,0.3,7.80


# Web scraping

I'ts not uncommon to find an HTML table on a web site with the data that we want. How can we retrieve the data from that table? Normally, we need "scrape" the web page, then grab the HTML table from within.

Pandas offers this to us automatically!  We just need to use the `pd.read_html` function.  Hand that a URL, and it returns a Python list of data frames. Every HTML table in the page is downloaded into a separate data frame.

You just need to identify which data frame is of interest, and then work on it.